# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data


A model that can estimate how much something costs, from its description.

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 2: Data Pre-processing

Today we'll rewrite the products into a standard format.  
LLMs are great at this!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of Data Pre-processing / Re-writing</h2>
            <span style="color:#181;">LLMs have made it simple to do something that was considered impossible only a few years ago.
            This approach can be applied to almost any business vertical, and it's similar to the advanced techniques
            we used on Week 5.</span>
        </td>
    </tr>
</table>

In [1]:
from litellm import completion
from dotenv import load_dotenv
import json
from pricer.batch import Batch
from pricer.items import Item

load_dotenv(override=True)

True

# The next cell is where you choose Dataset

Use `LITE_MODE = True` for the free, fast version with training data size of 20,000

USe `LITE_MODE =  False` for the powerful, full version with training data size of 800,000

## For this lab

You can skip altogether and load the dataset from HuggingFace: $0

You can run pre-processing for the lite dataset: under $1

You can run pre-processing for the full dataset: $30

In [2]:
LITE_MODE = True

In [3]:
username = "ed-donner"
dataset = f"{username}/items_raw_lite" if LITE_MODE else f"{username}/items_raw_full"

train, val, test = Item.from_hub(dataset)

items = train + val + test

print(f"Loaded {len(items):,} items")
print(items[0])

README.md:   0%|          | 0.00/738 [00:00<?, ?B/s]

d:\Crest-Training\10-LLM-Engineering\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PriyamVyas\.cache\huggingface\hub\datasets--ed-donner--items_raw_lite. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loaded 22,000 items
title='Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)' category='Tools_and_Home_Improvement' price=64.3 full='Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\n[\'From the Manufacturer\', "When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we\'re the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid"]\n[\'Interior half only\', \'Requires F58 to complete handle set\', \'Non handed knob style\', \'4" minimum center to center door prep required for this two piece model.\', \'Lifetime Mechanical and Finish Warranty\']\n{"Material": "Metal", "Brand": "", "Color": "Oil Rubbed Bronze", "Exterior Finish": "Bronze", "Special Feature": "Easy to Install", "Age Range (Description)": "Adult", "Included Components": "Deadbolt, Knob", "Item Wei

In [4]:
items[2].id

In [5]:
# Give every item an id

for index, item in enumerate(items):
    item.id = index

In [6]:


SYSTEM_PROMPT = """Create a concise description of a product. Respond only in this format. Do not include part numbers.
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features"""

In [7]:
print(items[0].full)

Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)
['From the Manufacturer', "When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we're the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid"]
['Interior half only', 'Requires F58 to complete handle set', 'Non handed knob style', '4" minimum center to center door prep required for this two piece model.', 'Lifetime Mechanical and Finish Warranty']
{"Material": "Metal", "Brand": "", "Color": "Oil Rubbed Bronze", "Exterior Finish": "Bronze", "Special Feature": "Easy to Install", "Age Range (Description)": "Adult", "Included Components": "Deadbolt, Knob", "Item Weight": "1.5 pounds", "Handle Material": "Bronze", "Package Type": "Standard Packaging", "Unit Count": "1.0 Count", "Number of Items": "1", "Manufacturer": "Schlage", "Product Dimensions": "8.1 x 4

In [8]:
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": items[0].full}]
response = completion(messages=messages, model="groq/openai/gpt-oss-20b", reasoning_effort="low")

print(response.choices[0].message.content)
print()
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Cost: {response._hidden_params['response_cost']*100:.3f} cents")


Title: Schlage F59 & 613 Interior Knob with Deadbolt – Oil Rubbed Bronze  
Category: Door Hardware  
Brand: Schlage  
Description: A durable, oil‑rubbed bronze interior knob paired with a deadbolt for enhanced security.  
Details: Features a solid metal construction, easy installation, and a lifetime mechanical and finish warranty.

Input tokens: 446
Output tokens: 93
Cost: 0.009 cents


In [9]:

messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": items[0].full}]
response = completion(messages=messages, model="ollama/llama3.2", api_base="http://localhost:11434")
print(response.choices[0].message.content)
print()
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Cost: {response._hidden_params['response_cost']*100:.3f} cents")


### Product:
**Smart Home Security System**
**Category: Electronics**
**Brand: Amazon**
Description: A comprehensive security system that integrates cameras, motion sensors, and smart door locks to provide real-time monitoring and control of your home.
Details: Features advanced AI-powered video analysis, voice command integration, and biometric authentication for enhanced security and convenience.

Input tokens: 406
Output tokens: 71
Cost: 0.000 cents


In [10]:
MODEL = "openai/gpt-oss-20b"


In [11]:
def make_jsonl(item):
    body = {"model": MODEL, "messages": [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": item.full}], "reasoning_effort": "low"}
    line = {"custom_id": str(item.id), "method": "POST", "url": "/v1/chat/completions", "body": body}
    return json.dumps(line)

In [12]:
items[0]

<Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only) = $64.3>

In [13]:
make_jsonl(items[0])

'{"custom_id": "0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "openai/gpt-oss-20b", "messages": [{"role": "system", "content": "Create a concise description of a product. Respond only in this format. Do not include part numbers.\\nTitle: Rewritten short precise title\\nCategory: eg Electronics\\nBrand: Brand name\\nDescription: 1 sentence description\\nDetails: 1 sentence on features"}, {"role": "user", "content": "Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\\n[\'From the Manufacturer\', \\"When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we\'re the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid\\"]\\n[\'Interior half only\', \'Requires F58 to complete handle set\', \'Non handed knob style\', \'4\\" minimum center to center door prep required for this two piece m

In [ ]:
import os

def make_file(start, end, filename):
    # Ensure directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    with open(filename, "w") as f:
        for i in range(start, end):
            f.write(make_jsonl(items[i]))
            f.write("\n")

In [32]:
make_file(0, 1000, "jsonl/0_1000.jsonl")